### Scraping website using pandas

### Import Pandas Library

In [7]:
import pandas as pd

### Use pd.read_html() to get all the tables on the website 

In [8]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = pd.read_html(url)

### We get a list of dataframe, if we have multiple tables on the website. Choose the required table from the list.

In [9]:
list_toronto = data[0]
list_toronto.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Ignoring rows with Not assigned values for column Borough .

In [10]:
list_toronto = list_toronto[list_toronto['Borough'] != 'Not assigned']
list_toronto.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Check the shape of the dataframe

In [11]:
list_toronto.shape

(103, 3)

### Install Geocoder

In [40]:
!pip install geocoder

     |████████████████████████████████| 102kB 1.7MB/s ta 0:00:011


### define method to get Coordinates

In [42]:
import geocoder


def get_location(postal_code):
    
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        
        return lat_lng_coords
    


### Getting Coordinates using Geocoder

In [ ]:
post_code = list(list_toronto['Postal Code'].unique())
latlog = {}
for i in post_code:
    coords = get_location(i)
    latlog[i]=coords

### Above took extremely long time. So, alternative csv was used.

In [12]:
location_data = pd.read_csv("/Users/SujayaJU/Downloads/Geospatial_Coordinates.csv") ## if, running the code please give the path where you have stored the file
location_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Using pd.merge() to join two dataframes based on Postal Code

In [23]:
list_toronto_latlog = pd.merge(list_toronto, location_data, on='Postal Code')
list_toronto_latlog.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Get Co-Ordinates of Toronto

In [14]:
# Toronto coordinates

latitude = 43.651070
longitudnal = -79.347015

### Using Folium to mark the neighbourhood on the Map

In [15]:
!pip install folium
import folium

In [21]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitudnal], zoom_start=11)

# add markers to map
for lat, lng, label in zip(list_toronto_latlog['Latitude'], list_toronto_latlog['Longitude'], list_toronto_latlog['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=True).add_to(map_toronto)  
    
map_toronto

#### There are more neighbourhoods closer to each other in Toronto City and near the ocean, compared to other neighbourhoods that are more spread out.